In [40]:
import numpy as np
import awkward as ak
import uproot
import matplotlib.pyplot as plt
import mplhep as hep
from scipy.optimize import curve_fit
from scipy.stats import norm, stats

import sys
sys.path.append('/home/users/hswanson13/tbanalysis/') #stupid pytho
import run_analysis as ra
from os import path
from datetime import datetime
import utils.analysis as au
from os import path
import uproot
import yahist

import utils.plotting as pu

In [41]:
base_path = "/home/users/hswanson13/tbanalysis/output_analysis_data/test_output6_parquet"
output_dir = "start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/"
#output_dir = "start_4600_stop_4899_bias_270_offset_15.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned"

# dataColl = ak.from_parquet(
#     path.join(base_path, output_dir, "fullCollection.parquet"
#     )
# )
dataPresel = ak.from_parquet(
    path.join(base_path, output_dir, "fullPresel.parquet"
    )
)

In [ ]:
from scipy.optimize import curve_fit
def gaus(x, *p0):
    N, mu, sigma = p0
    return N*np.exp(-(x-mu)**2/(2.0*sigma**2))

def plot_gauss_fit(ax, hist_counts, bin_centers, p0, color='red', lw=2):
    # Do a simple gaussian fit
    try:
        coeff, var_martix = curve_fit(gaus, bin_centers, hist_counts, p0=p0)
    except RuntimeError:
        coeff = p0
    ax.plot(
        bin_centers,
        gaus(bin_centers, *coeff),
        color=color,
        label='Gaussian Fit\n mean: {:.2f} \n sigma: {:.3f}'.format(coeff[1],abs(coeff[2])),
        linewidth=lw
    )

row, col = 9, 8

pix_sel = ((dataPresel.row==row) & (dataPresel.col==col))
#print(dataPresel['dt'][pix_sel])

fig = plt.figure(constrained_layout=True,  figsize=(14, 6))
ax = plt.subplot(1,1,1)

dt_hist = yahist.Hist1D(ak.flatten(dataPresel['dt_corr'][pix_sel]))
dt_hist.plot(ax, color='orange', label="dt")#label=f"dt Row={row}, Col={col}")
plot_gauss_fit(ax, dt_hist.counts, dt_hist.bin_centers, [np.max(dt_hist.counts),0, 0.1], lw=2)

ax.legend(loc=1, prop={'size': 18}, frameon=True,fancybox=True)

ax.set_ylabel("counts")
ax.set_xlabel("t (ns)")
ax.set_title("dt")



In [ ]:
# import importlib
# importlib.reload(pu)

dt_corr_hist = pu.get_data(path.join(base_path, output_dir, 'dt_corrected_hist.pkl'))

fig = pu.plot_corrected_dt_per_pixel(dt_corr_hist, 9,7)

